In [3]:
import numpy as np

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

tf.executing_eagerly()

True

In [2]:
# thanks https://stackoverflow.com/questions/34199233/how-to-prevent-tensorflow-from-allocating-the-totality-of-a-gpu-memory
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

2021-08-30 06:57:04.760653: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-08-30 06:57:04.780824: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2021-08-30 06:57:04.780848: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: winter
2021-08-30 06:57:04.780853: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: winter
2021-08-30 06:57:04.780933: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.57.2
2021-08-30 06:57:04.780957: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.57.2
2021-08-30 06:57:04.780961: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.57.2


In [4]:
Pa = np.array([[0.71893144, 0.14144765, 0.10698649, 0.03263443],
       [0.06325843, 0.77423272, 0.13928361, 0.02322523],
       [0.02322523, 0.13928361, 0.77423272, 0.06325843],
       [0.03263443, 0.10698649, 0.14144765, 0.71893144]])

Pb = np.array([[0.95154866, 0.01674797, 0.0234783 , 0.00822507],
       [0.03863562, 0.84803839, 0.04732459, 0.0660014 ],
       [0.0660014 , 0.04732459, 0.84803839, 0.03863562],
       [0.00822507, 0.0234783 , 0.01674797, 0.95154866]])

Pc = np.array([[0.88137945, 0.04613594, 0.02686734, 0.04561727],
       [0.05009116, 0.9236776 , 0.0194023 , 0.00682894],
       [0.00682894, 0.0194023 , 0.9236776 , 0.05009116],
       [0.04561727, 0.02686734, 0.04613594, 0.88137945]])
Pm = np.array([[0.81203158, 0.04399917, 0.07848442, 0.06548484],
       [0.08284662, 0.87115553, 0.02024715, 0.0257507 ],
       [0.0257507 , 0.02024715, 0.87115553, 0.08284662],
       [0.06548484, 0.07848442, 0.04399917, 0.81203158]])

pi = np.array([0.14977394, 0.26317362, 0.39165429, 0.19539816])

pi_m = pi @ Pm

J = np.array([[[0.07508613297288518, 0.00466350788803318, 0.0033432901131033796, 0.004042219322757277], [0.002186937387234338, 0.01570096614179908, 0.0011254388766303509, 0.0005015574893929194], [0.002007283482847851, 0.0012535337368050612, 0.013395860029779497, 0.0010220788989917808], [0.0013011086711092986, 0.0016042909684947522, 0.0012404869820179004, 0.011323417297654589]], [[0.03129698648252512, 0.00815953485072002, 0.004394876926278819, 0.0020308813605868174], [0.008288184321098279, 0.14201837079233715, 0.005398759767993272, 0.0017880581744532536], [0.0015277493099248596, 0.00886482820182737, 0.04269081732021271, 0.0028169181161841477], [0.0021015052010260205, 0.011819352252411985, 0.0034103593745217937, 0.02379160092199082]], [[0.02174460265532043, 0.002941403304710777, 0.017006110729978843, 0.0023681733984058486], [0.0022577498981608663, 0.03237071808690972, 0.012426756488990255, 0.0019160961759307586], [0.0022144999652547693, 0.006239396030284971, 0.20953281339937846, 0.012130269098422072], [0.002543170970124578, 0.003973434481032667, 0.011775618913078878, 0.04210892174683528]], [[0.012093125924573445, 0.0012993605630256348, 0.0021464967072447926, 0.0015645368675293004], [0.0010702747292501203, 0.013479197587703494, 0.0016493822206027895, 0.0025896804160072057], [0.0005894417935184289, 0.00127918145334602, 0.022260289874459077, 0.0029430040436876307], [0.005229012301832341, 0.0040480932067737535, 0.006152859243062461, 0.09785943409093556]]])

In [5]:
J_mb = pi_m[:, np.newaxis] * Pb
J_bc = J_mb.T @ Pc
J_ab = Pa.T @ np.diag(pi) @ Pm @ Pb
J_ca = (Pa.T @ np.diag(pi) @ Pm @ Pc).T

assert np.allclose(J_bc, J.sum(axis=0))  # checking I have constructed it correctly
assert np.allclose(J_ab, J.sum(axis=2))
assert np.allclose(J_ca, J.sum(axis=1).T)

J_einsum = np.einsum('i,ij,ik,ku,kv', pi, Pa, Pm, Pb, Pc)
assert np.allclose(J_einsum, J)

In [10]:
# thanks https://github.com/mlgxmez/thelongrun_notebooks/blob/master/MLE_tutorial.ipynb

@tf.function()
def transform(params):
    pi = tfb.SoftmaxCentered()(params[0])
    Pa = tfb.SoftmaxCentered()(params[1:5])
    Pm = tfb.SoftmaxCentered()(params[5:9])
    Pb = tfb.SoftmaxCentered()(params[9:13])
    Pc = tfb.SoftmaxCentered()(params[13:17])
    return pi, Pa, Pm, Pb, Pc
    
@tf.function()
def loss(params, data):
    pi, Pa, Pm, Pb, Pc = transform(params)
    J = tf.einsum('i,ij,ik,ku,kv', pi, Pa, Pm, Pb, Pc)
    loss = tf.reduce_sum(tf.keras.losses.KLDivergence()(J, data))
    return loss

def mle_run(data, loss, parameters, optimizer, steps=1000, verbose=False):
    update_list = []
    
    for i in range(steps):
        with tf.GradientTape() as tape:
            loss_value = loss(parameters, data)
        gradients = tape.gradient(loss_value, parameters)
        optimizer.apply_gradients([(gradients, parameters)])
    
        if i % 10 == 0:
            update_list.append((
                optimizer.iterations.numpy(),loss_value.numpy(), 
                parameters.numpy()))
            iter_info = f"Step: {optimizer.iterations.numpy()}, initial loss: {loss_value.numpy()}"
            if verbose:
                print(iter_info)
    
    return update_list

In [16]:
normal_initializer = tf.random_normal_initializer()
params = tf.Variable(normal_initializer(shape=[17, 3], dtype=tf.float32), name='params')

optimizer = tf.optimizers.Adam(0.01)
update_list = mle_run(J, loss, params, optimizer, steps=3000, verbose=True)

Step: 1, initial loss: 0.06791029125452042
Step: 11, initial loss: 0.06488090008497238
Step: 21, initial loss: 0.06276635825634003
Step: 31, initial loss: 0.061023350805044174
Step: 41, initial loss: 0.0595443993806839
Step: 51, initial loss: 0.05813857167959213
Step: 61, initial loss: 0.05645224079489708
Step: 71, initial loss: 0.05405370891094208
Step: 81, initial loss: 0.05076826363801956
Step: 91, initial loss: 0.046919479966163635
Step: 101, initial loss: 0.043074071407318115
Step: 111, initial loss: 0.0395585298538208
Step: 121, initial loss: 0.03618793934583664
Step: 131, initial loss: 0.03273250162601471
Step: 141, initial loss: 0.029466306790709496
Step: 151, initial loss: 0.026662997901439667
Step: 161, initial loss: 0.024276018142700195
Step: 171, initial loss: 0.022198786959052086
Step: 181, initial loss: 0.0204582829028368
Step: 191, initial loss: 0.019036371260881424
Step: 201, initial loss: 0.017928624525666237
Step: 211, initial loss: 0.017046235501766205
Step: 221, ini

Step: 1751, initial loss: 9.58787313720677e-06
Step: 1761, initial loss: 9.37754703045357e-06
Step: 1771, initial loss: 9.180257620755583e-06
Step: 1781, initial loss: 8.975812306744047e-06
Step: 1791, initial loss: 8.77881575434003e-06
Step: 1801, initial loss: 8.600445653428324e-06
Step: 1811, initial loss: 8.412662282353267e-06
Step: 1821, initial loss: 8.224800694733858e-06
Step: 1831, initial loss: 8.056363185460214e-06
Step: 1841, initial loss: 7.883759280957747e-06
Step: 1851, initial loss: 7.708665179961827e-06
Step: 1861, initial loss: 7.545745120296488e-06
Step: 1871, initial loss: 7.384719083347591e-06
Step: 1881, initial loss: 7.232208190544043e-06
Step: 1891, initial loss: 7.065909812808968e-06
Step: 1901, initial loss: 6.9183643063297495e-06
Step: 1911, initial loss: 6.77016669214936e-06
Step: 1921, initial loss: 6.620727162953699e-06
Step: 1931, initial loss: 6.479394414782291e-06
Step: 1941, initial loss: 6.333907094813185e-06
Step: 1951, initial loss: 6.198831215442624

In [17]:
for p, phat in zip([pi, Pa, Pm, Pb, Pc], transform(params)):
    print(p)
    print(phat.numpy())
    print()

[0.14977394 0.26317362 0.39165429 0.19539816]
[0.17649218 0.38592875 0.14713755 0.29044157]

[[0.71893144 0.14144765 0.10698649 0.03263443]
 [0.06325843 0.77423272 0.13928361 0.02322523]
 [0.02322523 0.13928361 0.77423272 0.06325843]
 [0.03263443 0.10698649 0.14144765 0.71893144]]
[[0.05230147 0.09148193 0.1839756  0.672241  ]
 [0.0419483  0.13268892 0.7509073  0.07445542]
 [0.64273643 0.14791214 0.13135116 0.07800025]
 [0.06818739 0.7274398  0.14446567 0.05990707]]

[[0.81203158 0.04399917 0.07848442 0.06548484]
 [0.08284662 0.87115553 0.02024715 0.0257507 ]
 [0.0257507  0.02024715 0.87115553 0.08284662]
 [0.06548484 0.07848442 0.04399917 0.81203158]]
[[0.01675984 0.01031547 0.01294123 0.95998347]
 [0.01417217 0.9158386  0.0149206  0.05506857]
 [0.92680544 0.01518238 0.02910416 0.02890802]
 [0.07432606 0.03245734 0.85027885 0.04293775]]

[[0.95154866 0.01674797 0.0234783  0.00822507]
 [0.03863562 0.84803839 0.04732459 0.0660014 ]
 [0.0660014  0.04732459 0.84803839 0.03863562]
 [0.0082